# 自然言語処理モデルーBERT説明

BERT[(Bidirectional Encoder Representations from Transformers)](https://arxiv.org/abs/1810.04805)は google さんが２０１８年１０月に研究開発されました。


* BERTのアーキテクチャーについて


* BERTの事前学習モデル 


* ディープラーニングのモデルについて、簡単に説明


* BERTのTutorial（文書の感情分析）を検証します。


* BERTに関して、その他情報


* 自然言語処理の学習リソースをご紹介








### BERTのアーキテクチャーについて

BERT BASE と BERT LARGE

<img src="bert-base-bert-large-encoders.png"  width=75% >

<img src="bert-output-vector.png" width=100% >

inputの部分はテキストから単語の単位で分割してトークン化にして、トークンした数字をモデルにinputします。

モデルの部分は事前学習したモデルを利用し、実装します。

outputの青い部分から、分類とか数値の予測とかFeed-forwardで結果を出力できます。



### BERTの事前学習モデル 

BERTの事前学習モデル pre-train modelは[huggingface.co](http://huggingface.co)サイトに数多く公表されています。


### ディープラーニングのモデルについて、簡単に説明


<img src="neuro_picture.png"  width=65% >

サンプルデーターセット｛X,Y｝がありますと。  <span style="color:red; font-size:3em;">f(x)=ax+b</span>  のファンクションの係数<span style="color:red; font-size:3em;">a,b</span>を解くことです。

\begin{equation}\label{eq:}
f(x)=y \\ \left\{\ x,y  | \forall x \in X, \forall y \in Y \right\}
\end{equation}


例えば、２つサンプルデーターがありますと、下記計算式の解は逆算できます。aとbのニューロン係数を解きます。。

\begin{cases}
    ax_1+b=y_1        & \left\{\ x_1,y_1  | x_1 \in X,  y_1 \in Y \right\} \\
    ax_2+b=y_2        & \left\{\ x_2,y_2  |  x_2 \in X,  y_2 \in Y \right\} \\
    \vdots            &  \vdots   \\
    ax_n+b=y_n        & \left\{\ x_n,y_n  |  x_n \in X,  y_n \in Y \right\} \\
\end{cases}


ニューラルネットワークの訓練は、バイオロジーと同じく実験に失敗してからの経験を重なって、この経験を利用し、係数を修正すると最後に正しくなることです。




### BERTのTutorial（文書の感情分析）を検証します。

ちょっと変わったら、顧客意見探知など出来ます。

日本語のBERTモデルです。

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, BertForSequenceClassification


In [2]:
tokenizer = AutoTokenizer.from_pretrained('daigo/bert-base-japanese-sentiment')

#####  Tokenizetion について、、、



In [3]:
tokenizer.tokenize('東京都、19日に休業要請を全面解除へ')

['東京', '都', '、', '19', '日', 'に', '休業', '要請', 'を', '全面', '解除', 'へ']

In [4]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize('東京都、19日に休業要請を全面解除へ'))

[391, 409, 6, 39, 32, 7, 16212, 4432, 11, 6060, 6140, 118]

In [6]:
tokenizer.tokenize('意味のないアラート発動だった。')

['意味', 'の', 'ない', 'アラ', '##ート', '発動', 'だっ', 'た', '。']

In [7]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize('意味のないアラート発動だった。'))

[967, 5, 80, 2416, 145, 9144, 308, 10, 8]

### Input data について

特別なトークン：[PAD] , [UNK] , [CLS] , [SEP] , [MASK]

In [8]:
tokenizer.encode('意味のないアラート発動だった。', add_special_tokens=True)

[2, 967, 5, 80, 2416, 145, 9144, 308, 10, 8, 3]

In [9]:
tokenizer.vocab_size

32000

In [ ]:
tokenizer.vocab.keys()

odict_keys(['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', 'の', '、', 'に', '。', 'は', 'た', 'を', 'で', 'と', 'が', 'し', 'て', '1', 'な', '年', 'れ', 'い', 'あ', '(', ')', '2', 'さ', 'こ', 'も', 'か', '##する', 'ある', '日', 'いる', 'する', '・', '「', '月', '」', '19', 'から', '20', '大', 'ア', 'そ', 'こと', '##して', 'ま', '3', 'や', 'として', '中', '一', '人', 'よ', 'ス', 'によ', '4', 'なっ', 'その', 'ら', '-', 'れる', '『', 'など', '』', 'フ', 'シ', '##リー', '同', 'この', '出', '時', 'お', '地', 'だ', '5', '行', '201', '国', 'ない', '的', 'ため', '後', 'られ', '発', '200', '##ール', 'イ', '##ラン', '作', '日本', '##ター', '##ック', '市', '戦', 'マ', '第', '自', 'コ', '以', '6', 'あっ', 'カ', '者', '開', 'また', '高', '本', '上', 'オ', '##ット', '学', '最', 'とい', '現', 'バ', 'サ', 'へ', '##スト', 'もの', '10', '東', '##ング', 'よう', '名', 'まで', '7', '生', '部', 'あり', '世', 'プ', 'エ', '8', '##ュー', '会', '主', 'ロ', 'なる', 'という', '多', '"', '.', '機', '##ート', '事', '新', 'デ', '県', 'ジ', '全', '##いて', '##ーム', '##って', '分', '選', 'ク', '##ティ', '9', '小', 'ド', '当', '18', '##ント', 'ト', 'ラ', '家', 'メ', '##ード', '公', 'これ', 'レ', '場', '前', 'チ', '初', '長', 'パ', '通', 'S', 'つ', 'リ', '入', 'C', 'キ', '内', '##かっ', '受', '特', 'にお', 'せ', 'A', '実', '##ンド', '連', '使', '12', '回', '軍', '##ョン', '代', '199', 'でき', '記', 'う', 'おり', '教', '設', 'ブ', '北', '##ディ', '見', '11', '文', 'テ', '対', '##イン', 'それ', '目', 'ハ', 'より', ')、', '平', '結', 'により', '経', 'タ', ',', '##リア', 'によって', '##00', 'ほ', '所', '関', '駅', 'M', '化', '##ース', '運', '三', '用', '##リカ', 'ウ', '放', '性', '番', '位', '政', 'モ', 'による', '下', 

### BERTのニューラルネットワークを作成、

In [11]:
model = BertForSequenceClassification.from_pretrained(
    "daigo/bert-base-japanese-sentiment",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,)

In [12]:
input_ids = torch.tensor(tokenizer.encode("気持ちいい", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
#labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
#outputs = model(input_ids, labels=labels)
input_ids

tensor([[    2,  8415, 28457, 28457,     3]])

In [13]:
outputs = model(input_ids)

In [14]:
outputs

(tensor([[ 1.9109, -2.1092]], grad_fn=<AddmmBackward>),)

softmax function で分類する。

In [15]:
np.exp(1.9109)/(np.exp(1.9109)+np.exp(-2.1092))

0.9823653921066929

In [16]:
input_ids = torch.tensor(tokenizer.encode("気持ち悪い", add_special_tokens=True)).unsqueeze(0)  # Batch size 1

In [17]:
outputs = model(input_ids)

In [18]:
outputs

(tensor([[-2.2444,  1.9429]], grad_fn=<AddmmBackward>),)

In [19]:
np.exp(1.9429)/(np.exp(-2.2444)+np.exp(1.9429))

0.9850399664465486

評価ファンクション

In [20]:
def sa_function(text):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    positive = outputs[0][0][0].item()
    negative = outputs[0][0][1].item()
    positive_p = np.exp(positive)/(np.exp(positive)+np.exp(negative))
    negative_p = np.exp(negative)/(np.exp(positive)+np.exp(negative))
    if positive_p>=negative_p:
        result = {"label" : "ポジティブ","score" : positive_p}
    else:
        result = {"label" : "ネガティブ","score" : negative_p}
            
    return result

### モデル分類の効果を試します。

アマゾンサイトでコメント　と　ヤフーニュースのコメントで、軽く試しましょう。

In [21]:
a = sa_function('capslockキーが使えなくなりました。ここまでひどいキーボードは初めてです。')
a

{'label': 'ネガティブ', 'score': 0.7423163061031355}

In [22]:
a = sa_function('絶対に嫌だ。気持ち悪い旦那とこれからも一緒にいようだなんて。子供もいるし旦那が収入あろうとしても複数の女とやって天狗の渡部は嫌だな')
a

{'label': 'ネガティブ', 'score': 0.8131462307951975}

In [25]:
sa_function('むしろ、コロナを知らない？名前が同じだからとああだこうだと言ってるバカ達は歴史も知らないのかね。バカは困る、ホントに。')

{'label': 'ネガティブ', 'score': 0.7500206617469946}

### BERTの一族に関して、その他情報

その他transformerのモデル：

   * ELMO(94M); BERT(340M); GPT-2(1542M); Megatron(8G); T5(11G); Turing NLG(17G) (weight numbers)係数の量

普通のGPUだったら、BERTギリギリを使える感じです。これ以上はほぼ無理です。しかもモデル性能が高い反面、悪用の懸念があるため、まだ事前学習モデルが未公開多いです。


Open AiさんのGPT-2モデルに関して、事前学習モデル（公表されていません。）の性能


https://talktotransformer.com/

ALBERT:

   * BERTの軽量モデル。ｰ>性能が落ちなく、モバイルなどリソースが限られる利用のケースです。

最近に発表したGPT-3モデル：　

   * 最も大きい性能が人間より良いモデルです。パラメーラー数も大きいです。





### 自然言語処理のコースをご紹介

また、学生さんや、技術者、NLPのバックグランドを知りたい人に

もっと細かく自然言語処理を勉強したいだったら、下記のコースは私のレコメンドです。

* スタンフォード大学2017年秋のNLPコース：
https://www.youtube.com/playlist?list=PL3FW7Lu3i5Jsnh1rnUwq_TcylNr7EkRe6

* Coursera:
https://www.coursera.org/learn/nlp-sequence-models

２つコースは最新のBERT技術は含めてないですが。どちらでも大事な基礎の部分です。なければBERTの理解はできません。




次回の内容紹介：

訓練データーセットのデーター前処理

ちょっとどうやってモデルを訓練しますかを説明します。

RTE(Recognizing Textual Entailment) -> 推論認識でのBERTモデルの紹介

